
In this notebook, we will try to create interactive histograms to visualize ISIs distributions for selected neurons.



# Setup

First we load our data. We mount the drive, since we have files with data on Google Drive. If you don't have the data, you might need to pull it from GitHub.

In [1]:
# @markdown Mount Google Drive

#mount google drive
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [1]:
# Open the derictory where your data is.
# That's where it is for me, but it can be different in your case.
%cd /content/gdrive/My Drive/NMA/nma_easy_cabbage

/content/gdrive/My Drive/NMA/nma_easy_cabbage


Use `git` to pull up-to-date code for the repo.

In [3]:
!git pull

remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11 (delta 4), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/mikeonly/nma_easy_cabbage
   e8e5845..dd25dec  master     -> origin/master
Updating e8e5845..dd25dec
Fast-forward
 analysis/extract_ISIs.py        | 133 ++++++++++++++++++++++++++++++++++++++++
 processed_data/session11_isi.h5 | Bin 20642928 -> 76982852 bytes
 processed_data/session12_isi.h5 | Bin 33204960 -> 0 bytes
 processed_data/session13_isi.h5 | Bin 14238112 -> 0 bytes
 processed_data/session1_isi.h5  | Bin 21166672 -> 0 bytes
 processed_data/session2_isi.h5  | Bin 17389944 -> 0 bytes
 processed_data/session3_isi.h5  | Bin 27623768 -> 0 bytes
 processed_data/session4_isi.h5  | Bin 19671168 -> 0 bytes
 processed_data/session5_isi.h5  | Bin 19199160 -> 0 bytes
 9 files changed, 133 insertions(+)
 create mode 100644 analysis/ext

In [2]:
#@markdown Imports

!pip install --upgrade tables

import pandas as pd
import numpy as np
import altair as alt

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


Run the cell below to define a special class for plotting histograms.

# Bokeh

Bokeh would be a one way to try plotting the histograms.

In [126]:
#@markdown Bokeh imports

# Bokeh imports
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file, curdoc
from bokeh.models import ColumnDataSource, HoverTool, Panel, Slider, CustomJS, Range1d, TapTool
from bokeh.models.widgets import Tabs
from bokeh.layouts import column, layout, row
from bokeh.palettes import Category20

# Run this if inside a notebook
output_notebook()

## Histogram 1

Now histrograms work, but you need to manually select a neuron to plot.

In [ ]:
#@markdown Define Bokeh class for plotting

class BokehHistogram():

    def __init__(self, colors=["SteelBlue", "Tan"], height=600, width=600):
        self.colors = colors
        self.height = height
        self.width = width

    def hist_hover(self, hist, edges, show_plot=True):
        hist_df = pd.DataFrame({"isis": hist,
                                 "left": edges[:-1],
                                 "right": edges[1:]})
        hist_df["interval"] = ["%f to %f" % (left, right) for left, 
                               right in zip(hist_df["left"], hist_df["right"])]


        src = ColumnDataSource(hist_df)
        plot = figure(plot_height = self.height, plot_width = self.width,
              title = "Histogram of ISIs distribution",
              x_axis_label = "ISI, s",
              y_axis_label = "Count")    
        plot.quad(bottom = 0, top = "isis",left = "left", 
            right = "right", source = src, fill_color = self.colors[0], 
            line_color = "black", fill_alpha = 0.7,
            hover_fill_alpha = 1.0, hover_fill_color = self.colors[1])

        hover = HoverTool(tooltips = [('Interval', '@interval'),
                                  ('Count', str("@" + "isis"))])
        plot.add_tools(hover)

        if show_plot == True:
            show(plot)
        else:
            return plot

    def histotabs(self, dataframe, features, log_scale=False, show_plot=False):
        hists = []
        for f in features:
            h = self.hist_hover(dataframe, f, log_scale=log_scale, show_plot=show_plot)
            p = Panel(child=h, title=f.capitalize())
            hists.append(p)
        t = Tabs(tabs=hists)
        show(t)

    def filtered_histotabs(self, dataframe, feature, filter_feature, log_scale=False, show_plot=False):
        hists = []
        for col in dataframe[filter_feature].unique():
            sub_df = dataframe[dataframe[filter_feature] == col]
            histo = self.hist_hover(sub_df, feature, log_scale=log_scale, show_plot=show_plot)
            p = Panel(child = histo, title=col)
            hists.append(p)
        t = Tabs(tabs=hists)
        show(t)

In [ ]:
h = BokehHistogram()

In [ ]:
# Load the data Inge provided us

bins_session11 = np.load('processed_data/bins_session11_b0.02.npy')

# it has shape (698, 125) — (number of neurons, number of bins)
number_of_neurons, number_of_bins = bins_session11.shape

# bin size is 0.02 s, so we can create bins edges
bin_size = 0.02
edges = np.arange(0, number_of_bins + 1) * bin_size

In [ ]:
neuron_number = 34

h.hist_hover(bins_session11[neuron_number, :], edges)

## Histogram 2

In [ ]:
# Load the big dataframe 
df = pd.read_hdf('processed_data/session11_isi.h5', 'table')

In [81]:
session_number = 11
bin_size = 0.05  # in s

binned_session11 = np.load(f'processed_data/bins_session{session_number}_b{bin_size}.npy')
raw_labels = np.load('processed_data/labels11.npy')

number_of_neurons, number_of_bins = binned_session11.shape
edges = np.arange(0, number_of_bins+1) * bin_size

# Get mapping from labels to integers
cortex_areas, indexed_by_area = np.unique(raw_labels, return_inverse=True)

In [123]:
#@markdown Load and perform t-SNE and PCA

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

X = binned_session11

# Perform PCA
pca_model = PCA(n_components=50) # Initializes PCA
pca_model.fit(X) # Performs PCA 
scores = pca_model.transform(X)

pca_component1 = scores[:, 0]
pca_component2 = scores[:, 1]

# Perform t-SNE
tsne_model = TSNE(n_components=2, perplexity=10, random_state=2020) 
embed = tsne_model.fit_transform(X)

tsne_component1 = embed[:, 0]
tsne_component2 = embed[:, 1]

In [147]:
#@markdown 😈🎤 MIC DROP 

# Specify plot size
w, h = (700, 600)

# Plot the model fit

colors = Category20[len(cortex_areas)]
brain_area_colors = [colors[i] for i in indexed_by_area]

s1 = ColumnDataSource(
    data={"tsne1": tsne_component1,
          "tsne2": tsne_component2,
          "color": brain_area_colors,
          "cortex_area": raw_labels,})

p1 = figure(plot_height=h, plot_width=w, title="", sizing_mode="scale_both", 
           background_fill_color="#fafafa", tools="reset,box_zoom,pan")
p1.circle(x="tsne1", y="tsne2", source=s1, radius=1.2, 
          fill_color='color', legend_field='cortex_area', fill_alpha=0.7, line_alpha=0)
p1.xaxis.axis_label = "t-SNE1"
p1.yaxis.axis_label = "t-SNE2"


# Plot the histogram
s2 = ColumnDataSource(
    data={
        "top": binned_session11[1, :],
        "left": edges[:-1],
        "right": edges[1:],})
p2 = figure(plot_height=h, plot_width=w, title="", 
           toolbar_location=None, sizing_mode="scale_both", 
           background_fill_color="#fafafa")
p2.quad(top="top", bottom=0, left="left", right="right", source=s2, 
       fill_color="navy", line_color=None, alpha=0.5, )

p2.y_range.start = 0
p2.xaxis.axis_label = 'ISIs, s'
p2.yaxis.axis_label = 'Normalized count'
p2.grid.grid_line_color = 'white'
p2.y_range = Range1d(0, 0.4)


###############################
# Define what happens on change

cb_click = CustomJS(
    args={"s1": s1, "s2": s2, "hist": binned_session11}, 
    code=
    """
    var ind = cb_obj.indices[0]
    var d1 = s1.data;
    var d2 = s2.data;
    var top = d2['top'];
    for (var i = 0; i < top.length; i++) {
        top[i] = hist[ind][i]
    }
    s2.change.emit();
    """)
p1.add_tools(TapTool(callback=cb_click))
s1.selected.js_on_change('indices', cb_click)


l = layout(row(p1, p2))

show(l)

In [109]:
#@markdown An attempt with a slider. This was my first working prototype.

# Create ColumnDataSource that will be used in the plot
source = ColumnDataSource(
    data={
        "top": binned_session11[1, :],
        "left": edges[:-1],
        "right": edges[1:],})


neuron_number_slider = Slider(title="Neuron number", start=0, 
                              end=number_of_neurons, step=1, value=1)

callback = CustomJS(
    args=dict(source=source, hist=binned_session11), 
    code="""
        var data = source.data;
        var nn = cb_obj.value
        var top = data['top']
        for (var i = 0; i < top.length; i++) {
            top[i] = hist[nn][i]
        }
        source.change.emit();
        """)
neuron_number_slider.js_on_change('value', callback)

p = figure(plot_height=600, plot_width=700, title="", 
           toolbar_location=None, sizing_mode="scale_both", 
           background_fill_color="#fafafa")

p.quad(top="top", bottom=0, left="left", right="right", source=source, 
       fill_color="navy", line_color=None, alpha=0.5, )

p.y_range.start = 0
p.xaxis.axis_label = 'ISIs, s'
p.yaxis.axis_label = 'Normalized count'
p.grid.grid_line_color = 'white'
p.y_range = Range1d(0, 0.4)
  
l = layout([column(neuron_number_slider, width=320)], [p])

curdoc().add_root(l)
show(l)

# Altair

Altair seems to be an easier library to plot data with better documentation. Let's try it next.

In [9]:
source = ColumnDataSource(df)

In [ ]:
neuron_select_slider = alt.binding_range(min=0, max=number_of_neurons, step=1)
select_neuron = alt.selection_single(fields=['neuron_number'], 
                                     bind=neuron_select_slider,
                                     init={'neuron_number': 0})

alt.Chart(data=df).mark_bar()

import altair as alt
from vega_datasets import data

source = data.population.url

pink_blue = alt.Scale(domain=('Male', 'Female'),
                      range=["steelblue", "salmon"])

slider = alt.binding_range(min=1900, max=2000, step=10)
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2000})

alt.Chart(source).mark_bar().encode(
    x=alt.X('sex:N', title=None),
    y=alt.Y('people:Q', scale=alt.Scale(domain=(0, 12000000))),
    color=alt.Color('sex:N', scale=pink_blue),
    column='age:O'
).properties(
    width=20
).add_selection(
    select_year
).transform_calculate(
    "sex", alt.expr.if_(alt.datum.sex == 1, "Male", "Female")
).transform_filter(
    select_year
).configure_facet(
    spacing=8
)